In [1]:
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Psychologia/Magisterka/magisterka Psyche/Mateusz_Kamyczura_Praca_Magisterska/Kody/Baza danych'
mABC_path = path + '/metryczkaABC'
mXYZ_path = path + '/metryczkaXYZ'
tABC_path = path + '/tabeleABC'
tXYZ_path = path + '/tabeleXYZ'
Klucz_path = path + '/Input'
Output_path = path + '/Output'
poprawneABC_path = Klucz_path + '/kluczABC.xlsx'
ppoprawneXYZ_path = Klucz_path + '/kluczXYZ.xlsx'

Mounted at /content/drive


##Wczytanie i połączenie danych


In [2]:
poprawneABC = pd.read_excel(Klucz_path + '/kluczABC.xlsx')
poprawneXYZ = pd.read_excel(Klucz_path + '/kluczXYZ.xlsx')

In [3]:
AGL_A2 = ["abccacb", "bacbcab", "abababcccab", "ababcabccab",
                  "abcabcabcabcab", "abcababcab", "acbcabcab", "abcbcabccbcab",
                  "abcabccab", "abcaabcaabcab", "aabcacbcaba", "ababcccababab",
                  "abccababccab", "abcabccabcb", "abcabab", "abababababab", "abcacaab",
                  "cabbbca", "accbacbab", "abcabcccab", "babcabcabccab", "ababab",
                  "abcccab", "abababab", "abbcab", "abccab", "baccb", "abababccabcab",
                  "abcabbab", "ababcabcab", ]

ALG_B2 = ["XYXYYYYYYYYXYZ", "XXXXZZZXY", "XXXXYZZZZXY", "XZZZXXXY", "XZXYZZZXY",
                  "XXYXXYXZZZZZZXY", "XXXXXXYYZZZZZXY", "XXYXYXZZZZZXY", "XXXXXXXZZZZZZXY",
                  "XYYXXZZZZZXY", "XYYYYZZZZXY", "XXXXXYYYZXZZXY", "XYXXYYZZZZZXY", "XYZXY",
                  "XYZYYYZZXYZ", "XYXYXYZYZZZXY", "XYYYZZZZZXY", "XXXYYXYZZZZZZXY", "XXXXYYZXYZZZZZY",
                  "XXZXY", "XXYZZXY", "XXYXZZZXY", "XYYXYXYZZZZZZXY", "XZZZXY", "YYYZZZXY", "XYXXXZZXXXZYXY",
                  "ZZXYXZZZXY", "XYXXZZZXY", "XXZZXY", "XXXZZXY", ]


In [4]:
poprawneABC['Ciąg'] = AGL_A2
poprawneXYZ['Ciąg'] = ALG_B2

In [5]:
def polacz_metryczki(path):
    # Funkcja łączy metryczki z folderu w jednego dużego DF
    dataframes = []

    # Przechodzenie przez wszystkie pliki w folderze
    for file in os.listdir(path):
        full_path = os.path.join(path, file)

        # Sprawdzenie rozszerzenia pliku i wczytanie go do Pandas
        if file.endswith(".csv"):
            df = pd.read_csv(full_path, index_col=0)  # Pierwsza kolumna jako indeks
            dataframes.append(df)
        elif file.endswith(".xlsx") or file.endswith(".xls"):
            df = pd.read_excel(full_path, index_col=0)  # Pierwsza kolumna jako indeks
            dataframes.append(df)

    # Jeśli nie znaleziono plików, zwracamy pusty DataFrame
    if not dataframes:
        return pd.DataFrame()

    # Konkatenujemy wszystkie DataFrame'y w jeden
    df_final = pd.concat(dataframes, ignore_index=False)  # Zachowujemy indeksy

    # Resetujemy indeks
    df_final.reset_index(drop=True, inplace=True)

    return df_final

In [6]:
metryczkiABC = polacz_metryczki(mABC_path)
metryczkiXYZ = polacz_metryczki(mXYZ_path)

In [7]:
def polacz_tabele(path: str) -> pd.DataFrame:
    """
    Funkcja łączy wszystkie arkusze z plików Excel w folderze 'path', dodając kolumnę 'Key'
    z nazwą arkusza, a następnie scala wszystkie arkusze w jedną tabelę.

    :param path: Ścieżka do folderu zawierającego pliki Excel
    :return: Połączony DataFrame
    """
    all_data = []

    for file in os.listdir(path):
        if file.endswith(".xlsx") or file.endswith(".xls"):
            file_path = os.path.join(path, file)
            xls = pd.ExcelFile(file_path)

            for sheet_name in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=sheet_name)
                df["Key"] = sheet_name  # Dodaj kolumnę z nazwą arkusza
                all_data.append(df)

    merged_df = pd.concat(all_data, ignore_index=True)
    return merged_df

In [8]:
tabeleXYZ = polacz_tabele(tXYZ_path)
tabeleABC = polacz_tabele(tABC_path)

In [9]:
tabeleABC = tabeleABC[~tabeleABC["Lp."].isin([4, 23])]

##Odsianie obserwacji odstających

In [10]:
def sprawdz_poprawnosc_odpowiedzi(tabele: pd.DataFrame, poprawne: pd.DataFrame) -> pd.DataFrame:
    """
    Funkcja łączy dwa DataFrame'y po kolumnie "Lp" i zwraca połączoną tabelę.
    Dodatkowo dodaje kolumny 'CzyPoprawna' oraz 'obróconyOSAD'.
    CzyPoprawna mówi o tym, czy dla danego badanego i danego pytania odpowiedź była poprawna
    :param tabele: Pierwszy DataFrame
    :param poprawne: Drugi DataFrame
    :return: Połączony DataFrame
    """
    merged_df = tabele.merge(poprawne, on="Lp.", how="inner")

    # Dodaj kolumnę 'CzyPoprawna'
    merged_df["CzyPoprawna"] = merged_df.apply(lambda row: 1 if (row["OSAD"] > 5 and row["G"] == 1) or (row["OSAD"] < 5 and row["G"] == 0)
                                                else 0 if row["OSAD"] == 5
                                                else -1, axis=1)

    # Dodaj kolumnę 'obróconyOSAD'
    merged_df["obróconyOSAD"] = merged_df.apply(lambda row: row["OSAD"] if row["G"] == 1 else 10 - row["OSAD"], axis=1)

    return merged_df


def policz_procenty(df):
    """
    Liczy procent poprawnych odpowiedzi i procent odpowiedzi zgodnych z zasadą (I/Z == 2),
    oraz dodaje kolumny 'mat', 'Wiek' i 'Kobieta' – sprawdzając ich spójność.

    Argumenty:
    df : pandas.DataFrame
        Dane z kolumnami: 'Key', 'CzyPoprawna', 'I/Z', 'mat', 'Wiek', 'Kobieta'

    Zwraca:
    pandas.DataFrame z kolumnami:
        'Key', 'ProcentPoprawnych', 'ProcentZasad', 'mat', 'Wiek', 'Kobieta'
    """

    # Filtrowanie odpowiedzi różnej od 0 (np. brak odpowiedzi)
    df_filtered = df[df["CzyPoprawna"] != 0]

    # Procent poprawnych odpowiedzi
    procent_poprawnych = df_filtered.groupby("Key")["CzyPoprawna"].apply(lambda x: (x == 1).mean() * 100)

    # Procent zasad (I/Z == 2)
    procent_zasad = df.groupby("Key")["I/Z"].apply(lambda x: (x == 2).mean() * 100)

    # Sprawdzamy, czy kolumna 'mat', 'Wiek' i 'Kobieta' mają unikalną wartość dla każdego Key
    for col in ['mat', 'Wiek', 'Kobieta']:
        values_per_key = df.groupby("Key")[col].nunique()
        if (values_per_key > 1).any():
            raise ValueError(f"Dla niektórych 'Key' kolumna '{col}' zawiera różne wartości!")

    # Pobieramy wartość 'mat', 'Wiek' i 'Kobieta' dla każdego Key (po jednej reprezentatywnej wartości)
    mat_per_key = df.groupby("Key")["mat"].first()
    wiek_per_key = df.groupby("Key")["Wiek"].first()
    kobieta_per_key = df.groupby("Key")["Kobieta"].first()

    # Łączymy wszystko
    wynik = pd.concat([procent_poprawnych, procent_zasad, mat_per_key, wiek_per_key, kobieta_per_key], axis=1).reset_index()
    wynik.columns = ["Key", "ProcentPoprawnych", "ProcentZasad", "mat", "Wiek", "Kobieta"]

    return wynik

def znajdz_odstajace_IQR(df):
    """
    Zwraca listę wartości 'Key' dla obserwacji odstających metodą IQR (1.5 * IQR).

    Argumenty:
    df : pandas.DataFrame
        Tabela z danymi, gdzie:
        - "Key" to indeksy ludzi
        - "ProcentPoprawnych" to wynik procentowy

    Zwraca:
    list : lista wartości 'Key' dla obserwacji odstających
    """

    # Kopia danych
    df = df.copy()

    # Obliczamy kwartyle i IQR
    Q1 = df['ProcentPoprawnych'].quantile(0.25)
    Q3 = df['ProcentPoprawnych'].quantile(0.75)
    IQR = Q3 - Q1

    # Granice dla wartości odstających
    dolna_granica = Q1 - 1.5 * IQR
    gorna_granica = Q3 + 1.5 * IQR

    # Wyszukujemy wartości odstające
    maska_odstajace = (df['ProcentPoprawnych'] < dolna_granica) | (df['ProcentPoprawnych'] > gorna_granica)
    usuniete = df.loc[maska_odstajace, 'Key'].tolist()

    print(f"Metoda odfiltrowania: IQR")
    if usuniete:
        print(f"Znalezione obserwacje odstające (Key): {usuniete}")
    else:
        print("Nie znaleziono obserwacji odstających.")

    return usuniete


In [11]:
tabeleABCuzup = sprawdz_poprawnosc_odpowiedzi(tabeleABC, poprawneABC)
tabeleABCuzup = pd.merge(tabeleABCuzup, metryczkiABC, on="Key")
ProcentPoprwnychABC = policz_procenty(tabeleABCuzup)
OdstajaceABC = znajdz_odstajace_IQR(ProcentPoprwnychABC)
OdstajaceABC

Metoda odfiltrowania: IQR
Znalezione obserwacje odstające (Key): ['Do1']


['Do1']

In [12]:
tabeleXYZuzup = sprawdz_poprawnosc_odpowiedzi(tabeleXYZ, poprawneXYZ)
tabeleXYZuzup = pd.merge(tabeleXYZuzup, metryczkiXYZ, on="Key")
ProcentPoprwnychXYZ = policz_procenty(tabeleXYZuzup)
OdstajaceXYZ = znajdz_odstajace_IQR(ProcentPoprwnychXYZ)
OdstajaceXYZ

Metoda odfiltrowania: IQR
Znalezione obserwacje odstające (Key): ['S2']


['S2']

In [13]:
tabeleABC_odfiltrowane = tabeleABCuzup[~tabeleABCuzup['Key'].isin(OdstajaceABC)]
tabeleXYZ_odfiltrowane = tabeleXYZuzup[~tabeleXYZuzup['Key'].isin(OdstajaceXYZ)]

In [14]:
ABC = policz_procenty(tabeleABC_odfiltrowane)
XYZ = policz_procenty(tabeleXYZ_odfiltrowane)

In [15]:
All = pd.concat([ABC, XYZ], ignore_index=True)

In [16]:
All["Wiek"].min()

17

In [17]:
All["Wiek"].max()

55

In [18]:
All['Wiek'].mean()

np.float64(23.265060240963855)

In [19]:
All['Wiek'].std()

5.723473994830194

In [20]:
All['Kobieta'].sum()

np.int64(41)

In [21]:
len(All) - All['Kobieta'].sum()

np.int64(42)

##Przygotowanie danych pod analizę porównawczą matematyków i niematematyków


---



In [22]:
def grupowanie_po_wykształceniu(ABC, XYZ):
    # Dodanie sufiksu do kolumny "Key"
    ABC = ABC.copy()
    XYZ = XYZ.copy()

    ABC["Key"] = ABC["Key"].astype(str) + "_ABC"
    XYZ["Key"] = XYZ["Key"].astype(str) + "_XYZ"

    # Łączenie tabel
    df = pd.concat([ABC, XYZ], ignore_index=True)

    # Podział na "mat" i "niemat"
    mat = df[df["mat"] == 1].reset_index(drop=True)
    niemat = df[df["mat"] == 0].reset_index(drop=True)

    return mat, niemat

In [23]:
mat, niemat = grupowanie_po_wykształceniu(ABC,XYZ)

In [24]:
mat.to_excel(Output_path + '/mat.xlsx', index=False)
niemat.to_excel(Output_path + '/niemat.xlsx', index=False)

##Przygotowanie danych pod analizę porównawczą modeli


---



In [25]:
ABC.to_excel(Output_path + '/ABC.xlsx', index=False)
XYZ.to_excel(Output_path + '/XYZ.xlsx', index=False)

In [26]:
def wypisz_odpowiedzi_z_procentem(df1, df2):
    # Grupowanie danych po kolumnie "Lp."
    grouped = df1.groupby('Lp.')['CzyPoprawna'].value_counts().unstack(fill_value=0)

    # Dodanie kolumny "procent_poprawnych"
    grouped['procent_poprawnych'] = grouped[1] / (grouped[1] + grouped[-1]) * 100

    return grouped.merge(df2, on="Lp.")

In [27]:
PoprawnoscPytanABC = wypisz_odpowiedzi_z_procentem(tabeleABC_odfiltrowane,poprawneABC)
PoprawnoscPytanXYZ = wypisz_odpowiedzi_z_procentem(tabeleXYZ_odfiltrowane,poprawneXYZ)

In [28]:
PoprawnoscPytanABC.to_excel(Output_path + '/PoprawnoscPytanABC.xlsx', index=False)
PoprawnoscPytanXYZ.to_excel(Output_path + '/PoprawnoscPytanXYZ.xlsx', index=False)

##Przygotowanie danych pod analizę pewności osądu gramatycznego
---



In [29]:
pewnoscABC = tabeleABC_odfiltrowane[["Lp.", "CzyPoprawna", "mat",  "obróconyOSAD", "OSAD", "G", "Key"]]
pewnoscXYZ = tabeleXYZ_odfiltrowane[["Lp.", "CzyPoprawna", "mat",  "obróconyOSAD", "OSAD", "G", "Key"]]

In [30]:
pewnoscABC.to_excel(Output_path + '/pewnoscABC.xlsx', index=False)
pewnoscXYZ.to_excel(Output_path + '/pewnoscXYZ.xlsx', index=False)

##Przygotowanie danych pod analizę odwoływania się do zasad i intuicji


---



In [31]:
zasadyABC = tabeleABC_odfiltrowane[["Lp.", "CzyPoprawna", "I/Z", "mat"]]
zasadyXYZ = tabeleXYZ_odfiltrowane[["Lp.", "CzyPoprawna", "I/Z", "mat"]]

In [32]:
zasadyABC.to_excel(Output_path + '/zasadyABC.xlsx', index=False)
zasadyXYZ.to_excel(Output_path + '/zasadyXYZ.xlsx', index=False)